# IMPORTS

In [ ]:
!pip install nsepy
!pip install pandas
!pip install numpy
!pip install TA-Lib
!pip install plotly
!pip install yfinance

In [ ]:
from nsepy import get_history
from datetime import date
import pandas as pd
import numpy as np
import talib
import datetime
import yfinance as yf
import plotly.graph_objects as go

# FETCHING DATA FROM NSE

In [ ]:
scrip = 'HDFCAMC'

In [ ]:

stock_df = get_history(symbol=scrip,
                    start=date(2021,1,1), 
                    end=date(2021,12,31)) 

In [ ]:
pd.set_option('display.width', 10000)
print(f"Stock data size:{stock_df.shape}")
display(stock_df.head(2))
display(stock_df.tail(2))

# STOCK ANALYSIS

## CANDLESTICK CHART

In [ ]:
import plotly.graph_objects as go

candlestick = go.Candlestick(x=stock_df.index, 
                             open=stock_df['Open'],  
                             high=stock_df['High'], 
                             low=stock_df['Low'], 
                             close=stock_df['Close'])

fig = go.Figure(data=[candlestick])
fig.layout.xaxis.type = 'category' 
fig.show()

# CANDLESTICK PATTERNS

In [ ]:
def plot_candle(df, title):
    candlestick = go.Candlestick(x=df.index, open=df['Open'], high=df['High'], low=df['Low'], close=df['Close'])

    # Plot only the last month's data
    fig = go.Figure(data=[candlestick],
                    layout=go.Layout(title=go.layout.Title(text=title)))
    fig.layout.xaxis.type = 'category' 
    fig.show()


## MARUBOZU CANDLE - SINGLE CANDLESTICK PATTERN

In [ ]:
# Identify the marubozu candles in the dataset
stock_df['marubozu'] = talib.CDLMARUBOZU(stock_df['Open'], stock_df['High'], stock_df['Low'], stock_df['Close'])

# Subset dataframe for only the marubozu candles
marubozu_candles = stock_df[stock_df['marubozu'] != 0]
display(marubozu_candles[['Close','marubozu']])

# Plot the candlestick chart
plot_candle(marubozu_candles, title = f"Marubozu Candlestick Pattern - {scrip}")


## DOJI CANDLE

In [ ]:
# Identify the doji candles in the dataset
stock_df['doji'] = talib.CDLDOJI(stock_df['Open'], stock_df['High'], stock_df['Low'], stock_df['Close'])

# Subset dataframe for only the doji candles
doji_candles = stock_df[stock_df['doji'] != 0]
display(doji_candles[['Open','Close','doji']].tail())


# Plot the candlestick chart
plot_candle(doji_candles, title = f"Doji Candlestick Pattern - {scrip}")


## ENGULFING CANDLE

In [ ]:
# Identify the engulfing candles in the dataset
stock_df['engulf'] = talib.CDLENGULFING(stock_df['Open'], stock_df['High'], stock_df['Low'], stock_df['Close'])

# Subset dataframe for only the doji candles
stock_df['next_is_engulf'] = stock_df['engulf'].shift(-1)
engulf_candles = stock_df[(stock_df['engulf'] != 0) | (stock_df['next_is_engulf'] != 0)]
display(engulf_candles[['Open','Close','engulf']].head(10))


# Plot the candlestick chart
plot_candle(engulf_candles, title = f"Engulfing Candlestick Pattern - {scrip}")


## MORNING STAR/EVENING STAR CANDLE

In [ ]:
stock_df[stock_df['eve_star'] != 0]

In [ ]:
# Identify the morning/evening candles in the dataset
stock_df['morn_star'] = talib.CDLMORNINGSTAR(stock_df['Open'], stock_df['High'], stock_df['Low'], stock_df['Close'])           
stock_df['eve_star']  = talib.CDLEVENINGSTAR(stock_df['Open'], stock_df['High'], stock_df['Low'], stock_df['Close'])
stock_df['star'] = stock_df['morn_star'] + stock_df['eve_star'] 
# Subset dataframe for only the doji candles
stock_df['next_1_star'] = stock_df['star'].shift(-1, fill_value=0)
stock_df['next_2_star'] = stock_df['star'].shift(-2, fill_value=0)
star_candles = stock_df[(stock_df['star'] != 0) | (stock_df['next_1_star'] != 0) | (stock_df['next_2_star'] != 0)]
display(star_candles[['Close','morn_star','eve_star']].head(10))


# Plot the candlestick chart
plot_candle(star_candles, title = f"Morning/Evening Candlestick Pattern - {scrip}")


# TRADING STRATEGIES/INDICATORS

## MOVING AVERAGE

### SIMPLE MOVING AVERAGE

In [ ]:
stock_df['20_sma'] = stock_df['Close'].rolling(window=20).mean()
stock_df[['Close','20_sma']].tail()

### MOVING AVERAGE CHART

In [ ]:
# Compute the moving averages for 20 days, 50 days and 200 days
stock_df['20_sma'] = stock_df['Close'].rolling(window=20).mean()
stock_df['50_sma'] = stock_df['Close'].rolling(window=50).mean()
stock_df['200_sma'] = stock_df['Close'].rolling(window=200).mean()

# Plot the moving averages along with the candlestick chart
small_df = stock_df.tail(40)
candlestick = go.Candlestick(x=small_df.index, open=small_df['Open'], high=small_df['High'], low=small_df['Low'], close=small_df['Close'])
sma_20 = go.Scatter(x=small_df.index, y=small_df['20_sma'], name='20 days MA', line={'color': 'blue'})
sma_50 = go.Scatter(x=small_df.index, y=small_df['50_sma'], name='50 days MA', line={'color': 'orange'})
sma_200 = go.Scatter(x=small_df.index, y=small_df['200_sma'], name='200 days MA', line={'color': 'brown'})

# Plot only the last month's data
fig = go.Figure(data=[candlestick,sma_20,sma_50,sma_200],
                layout=go.Layout(title=go.layout.Title(text=f"MOVING AVERAGE CHART - {scrip}")))
fig.layout.xaxis.type = 'category' 
fig.show()


## RSI

In [ ]:
import talib
RSI_PERIOD = 14
stock_df['rsi'] = talib.RSI(stock_df['Close'], RSI_PERIOD) 
stock_df['rsi_indicator'] = stock_df['rsi'].apply(lambda x: 'BUY' if x<30 else 'SELL' if x>70 else 'HOLD')
display(stock_df[['Close','rsi','rsi_indicator']].tail(15))

## MACD

In [ ]:
scrip = 'HINDUNILVR'
stock_df = get_history(symbol=scrip,
                    start=date(2021,1,1), 
                    end=date(2021,12,31))

In [ ]:
from plotly.subplots import make_subplots

def plot_macd(stock_df, macd, signal, hist):
    fig = make_subplots( rows=2, cols=1)

    candlestick = go.Candlestick(x=stock_df.index, 
                                 open=stock_df['Open'],  
                                 high=stock_df['High'], 
                                 low=stock_df['Low'], 
                                 close=stock_df['Close'], name = 'daily candle')

    positive_hist = hist[hist>0]
    negative_hist = hist[hist<0]

    macd_line = go.Scatter(x=macd.index, y = macd, name = 'macd',line_color = 'blue')
    signal_line = go.Scatter(x=signal.index, y = signal, name = 'signal',line_color = 'orange')
    pos_hist_bar = go.Bar(x=positive_hist.index, y = positive_hist, name = 'hist',marker={'color': 'green'})
    neg_hist_bar = go.Bar(x=negative_hist.index, y = negative_hist, name = 'hist',marker={'color': 'red'})

    fig.add_trace(candlestick, row=1,col=1)    
    fig.add_trace(macd_line, row=2, col=1)
    fig.add_trace(signal_line, row=2, col=1)
    fig.add_trace(pos_hist_bar, row=2, col=1)
    fig.add_trace(neg_hist_bar, row=2, col=1)

    fig.update_layout(title_text=f'MACD - {scrip}', title_x=0.5,
                     xaxis_type = 'category',xaxis_rangeslider_visible = False,
                     xaxis_showticklabels = False,
                     xaxis2_type = 'category' )
    
    fig.show()


In [ ]:
import talib

MACD_FAST_EMA = 12
MACD_SLOW_EMA = 26
MACD_SIGNAL_PERIOD = 9 

macd, signal, hist = talib.MACD(stock_df['Close'], 
                                fastperiod=MACD_FAST_EMA, slowperiod=MACD_SLOW_EMA, signalperiod=MACD_SIGNAL_PERIOD)

plot_macd(stock_df, macd, signal, hist)

## BOLLINGER BAND

In [ ]:
scrip = 'ITC'
stock_df = get_history(symbol=scrip,
                    start=date(2021,1,1), 
                    end=date(2021,12,31))

In [ ]:
# 20-day moving average 
stock_df['sma_bb'] = stock_df['Close'].rolling(window=20).mean()

# upper and lower bollinger bands: SMA +/- 2 * standard deviation
stock_df['stddev'] = stock_df['Close'].rolling(window=20).std()
stock_df['upper_bb'] = stock_df['sma_bb'] + (2 * stock_df['stddev'])
stock_df['lower_bb'] = stock_df['sma_bb'] - (2 * stock_df['stddev'])

display(stock_df[['Close','sma_bb','upper_bb','lower_bb']]).tail()


# TALib - Bollinger Band
# upperband, middleband, lowerband = BBANDS(stock_df['Close'], timeperiod=20, nbdevup=2, nbdevdn=2, matype=0)


In [ ]:
def plot_bollinger_band(stock_df):

    candlestick = go.Candlestick(x=stock_df.index, 
                                 open=stock_df['Open'],  
                                 high=stock_df['High'], 
                                 low=stock_df['Low'], 
                                 close=stock_df['Close'], name = 'Daily candle')


    upper_line = go.Scatter(x=stock_df.index, y = stock_df['upper_bb'], name='upper_band',line_color = 'blue')
    lower_line = go.Scatter(x=stock_df.index, y = stock_df['lower_bb'], name = 'lower_band',line_color = 'blue')
    sma_line = go.Scatter(x=stock_df.index, y = stock_df['sma_bb'], name = 'sma_line',line_color = 'lightblue')   

                                                         
    fig = go.Figure(data=[candlestick, upper_line, lower_line, sma_line])
                                                         
    fig.update_layout(title_text=f'Bollinger Band - {scrip}', title_x=0.5,
                     xaxis_type = 'category',xaxis_rangeslider_visible = False,
                     xaxis_showticklabels = True )
    
    fig.show()
    
plot_bollinger_band(stock_df)

# BUILDING A SIMPLE TRADING STRATEGY

TRADING STRATEGY:  
1. Get the individual BUY/SELL/HOLD indicators based on the following strategies: Moving Average, MACD, Engulfing Pattern  
2. If even one of the strategies provide a BUY/SELL recommendation, then check the RSI value
3. If more than one indicator gives a signal, choose in this order: MACD, Moving Average, Engulfing Pattern  
4. RSI value of last 3 sessions considered
5. If strategy indicates a BUY and RSI < lower_threshold (40) --> Final Indicator = 'BUY'  
6. If strategy indicates a BUY and RSI > upper_threshold (60) --> Final Indicator = 'SELL' 


In [ ]:
# Get additional data to calculate the 200-d moving average from 1st Jan
start_date = date(2021,1,1) - datetime.timedelta(300) 
def fetch_data(scrip_name):
    return yf.Ticker(scrip_name).history(
                    start=start_date, 
                    end=date(2021,12,31))

In [ ]:
# Compute RSI
def apply_rsi_strategy(stock_df, rsi_period = 14):
    stock_df['rsi'] = talib.RSI(stock_df['Close'], rsi_period) 
    return stock_df

In [ ]:
# BUY/SELL/HOLD indicator based on the 200-days, 50-days and 20-days simple moving average
def get_ma_indicator(row):
    indicator = 'HOLD'
    row_values = [row['Close'], row['20_sma'], row['50_sma'] , row['200_sma']]
    
    # If close < 20-day sma < 50-day sma < 200-day sma --> BUY
    # If close > 20-day sma > 50-day sma > 200-day sma --> SELL
    if(row_values == sorted(row_values)):
        indicator = 'BUY'
    elif (row_values == sorted(row_values, reverse=True)):
        indicator = 'SELL'
        
    return indicator

def apply_moving_average_strategy(stock_df):
    # Compute the moving averages for 20 days, 50 days and 200 days
    stock_df['20_sma'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_sma'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['200_sma'] = stock_df['Close'].rolling(window=200).mean()

    stock_df['MA_indicator'] = stock_df.apply(lambda row: get_ma_indicator(row), axis=1)
    return stock_df
        

In [ ]:
# BUY/SELL/HOLD indicator based on the macd crossover

def get_macd_indicator(row):
    last_hist = row['hist']
    prev_hist = row['prev_hist']
    indicator = 'HOLD'
    # Provide BUY or SELL indications during crossovers
    if not np.isnan(prev_hist) and not np.isnan(last_hist):
        # If hist value has changed from negative to positive or vice versa, it indicates a crossover
        macd_crossover = (abs(last_hist + prev_hist)) != (abs(last_hist) + abs(prev_hist))
        if macd_crossover:
            indicator = 'BUY' if last_hist > 0 else 'SELL'
            
    return indicator
        
def apply_macd_strategy(stock_df):
    stock_df['macd'],  stock_df['signal'],  stock_df['hist'] = talib.MACD(stock_df['Close'], 
                                fastperiod=12, slowperiod=26, signalperiod=9)
    stock_df['prev_hist'] = stock_df['hist'].shift(1)
    stock_df['macd_indicator'] = stock_df.apply(lambda row: get_macd_indicator(row), axis=1)
    
    return stock_df
    

In [ ]:
# BUY/SELL/HOLD indicator based on the engulfing pattern

def apply_engulfing_pattern(stock_df):
    stock_df['engulf'] = talib.CDLENGULFING(stock_df['Open'], stock_df['High'], stock_df['Low'], stock_df['Close'])
    stock_df['engulf_indicator'] = stock_df.apply(lambda row: 'BUY' if (row['engulf'] > 0) else 'SELL' if (row['engulf'] < 0) else 'HOLD', axis=1)
    return stock_df



In [ ]:
CHECK_RSI_LAST_SESSIONS = 3
RSI_LOWER_LIMIT = 40
RSI_UPPER_LIMIT = 60
def apply_trading_algo(stock_df):
    stock_df['indicator'] = 'HOLD'
    buy_sell_signals = stock_df[(stock_df['macd_indicator'] != 'HOLD') | (stock_df['MA_indicator'] != 'HOLD')| (stock_df['engulf_indicator'] != 'HOLD')]
    for index,row in buy_sell_signals.iterrows():
        # Get previous n sessions whenever a signal is reached 
        analysis_df = stock_df.loc[:index].tail(3)
        macd_ind = analysis_df.iloc[-1]['macd_indicator']
        ma_ind = analysis_df.iloc[-1]['MA_indicator']
        engulf_ind = analysis_df.iloc[-1]['engulf_indicator']
        current_indicator = 'HOLD' 
        
        # Preference given in this order: MACD, MA, engulfing pattern
        current_indicator = macd_ind if macd_ind != 'HOLD' else \
                            ma_ind if ma_ind != 'HOLD' else engulf_ind
        
        # Decide the final indicator based on the RSI value
        if ((current_indicator == 'BUY') and (analysis_df['rsi'].min() <= RSI_LOWER_LIMIT)) \
            or \
            ((current_indicator == 'SELL') and (analysis_df['rsi'].max() >= RSI_UPPER_LIMIT)): 
            stock_df.at[index, 'indicator'] = current_indicator
        
    return stock_df



In [96]:
def plot_buy_sell_chart(scrip_name, stock):
    print("inside this method")
    candlestick = go.Candlestick(x=stock.index, 
                                 open=stock['Open'],  
                                 high=stock['High'], 
                                 low=stock['Low'], 
                                 close=stock['Close'])
    # print(candlestick)

    fig = go.Figure(data=[candlestick],
                   layout=go.Layout(title=go.layout.Title(text=f"TRADING ALGO - BUY/SELL RECOMMENDATIONS - {scrip_name}")))
    fig.layout.xaxis.type = 'category' 
    fig.layout.xaxis.rangeslider.visible = False

    for index, row in stock.iterrows(): 
        if (row['indicator'] == 'BUY'):
            print(row)
        if (row['indicator'] != 'HOLD'):
            line_colour = 'orange' if (row['indicator'] == 'BUY') else 'purple'
            fig.add_vline(x=row.name, line_width=3, line_dash="dash", line_color=line_colour)

            
    fig.show()
    
    

In [ ]:
def my_trading_algo(scrip_name):
    # Fetch the NSE data
    stock = fetch_data(scrip_name)

    # Apply the strategies individual
    stock = apply_moving_average_strategy(stock)
    stock = apply_rsi_strategy(stock)
    stock = apply_macd_strategy(stock)
    stock = apply_engulfing_pattern(stock)

    # Get only 2021 data for further analysis
    # import pdb; pdb.set_trace()
    stock = stock[stock.index >= str(date(2021,1,1))]

    # stock = print(stock)

    # Merge the strategies to create a final BUY/SELL/HOLD indicator
    stock = apply_trading_algo(stock)

    # # Plot the buy/sell recommendations on 2021 data
    plot_buy_sell_chart(scrip_name, stock)
    
    return stock

In [ ]:
hul = my_trading_algo('HINDUNILVR')

In [97]:
itc = my_trading_algo('ITC.NS')

inside this method
Open                207.349692
High                211.321013
Low                 206.789043
Close               210.666916
Volume                15904846
Dividends                  0.0
Stock Splits               0.0
20_sma              214.327555
50_sma              219.354313
200_sma              199.47581
MA_indicator              HOLD
rsi                  45.022243
macd                 -2.414117
signal               -1.485183
hist                 -0.928934
prev_hist            -1.207687
macd_indicator            HOLD
engulf                     100
engulf_indicator           BUY
indicator                  BUY
Name: 2021-12-02 00:00:00+05:30, dtype: object


In [ ]:
hdfcamc = my_trading_algo('HDFCAMC')

In [ ]:
whirlpool = my_trading_algo('WHIRLPOOL')

In [99]:
infy = my_trading_algo('INFY') # RSI is never less than 40 for INFY - hence no BUY calls

inside this method


In [100]:
#  FOR DEBUGGING
infy[['Symbol','Close','indicator','rsi','MA_indicator','macd_indicator','engulf_indicator']]

KeyError: "['Symbol'] not in index"

# OTHERS

## MORNING & EVENING STAR

In [ ]:

stock_df = get_history(symbol='DMART',
                    start=date(2021,1,1), 
                    end=date(2021,12,31))

In [ ]:
# Identify the morning/evening candles in the dataset
stock_df['morn_star'] = talib.CDLMORNINGSTAR(stock_df['Open'], stock_df['High'], stock_df['Low'], stock_df['Close'])           
stock_df['eve_star']  = talib.CDLEVENINGSTAR(stock_df['Open'], stock_df['High'], stock_df['Low'], stock_df['Close'])
stock_df['star'] = stock_df['morn_star'] + stock_df['eve_star'] 
# Subset dataframe for only the doji candles
stock_df['next_1_star'] = stock_df['star'].shift(-1, fill_value=0)
stock_df['next_2_star'] = stock_df['star'].shift(-2, fill_value=0)
star_candles = stock_df[(stock_df['star'] != 0) | (stock_df['next_1_star'] != 0) | (stock_df['next_2_star'] != 0)]
display(star_candles[['Close','morn_star','eve_star']])


# Plot the candlestick chart
plot_candle(star_candles, title = f"Morning/Evening Candlestick Pattern - DMART")


In [ ]:
plot_candle(stock_df.loc[date(2021,3,26):date(2021,5,10)], title = f"DMART - MORNING CANDLE")
plot_candle(stock_df.loc[date(2021,8,10):date(2021,9,15)], title = f"DMART - EVENING CANDLE")